In [1]:
import numpy as np

import pandas as pd
import sklearn
from sklearn import preprocessing

import bokeh
#from bokeh.charts import BoxPlot
import bokeh.plotting
from bokeh.plotting import ColumnDataSource
from bokeh.models import LabelSet
from bokeh.models import FuncTickFormatter
from bokeh.io import export_svgs

import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#load each run into a separate dataframe
df1 = pd.read_excel('190926_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df2 = pd.read_excel('191007_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df3 = pd.read_excel('191010_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df4 = pd.read_excel('191015_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df5 = pd.read_excel('191020_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df6 = pd.read_excel('191021_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df7 = pd.read_excel('191022_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df8 = pd.read_excel('191025_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df9 = pd.read_excel('191027_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df10 = pd.read_excel('191031_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df11 = pd.read_excel('191106_Deuterium_Transfer_Peak_Areas.xlsx', comment='#')
df12 = pd.read_csv('191115_Deuterium_Transfer_Peak_Areas.csv', comment='#')
df13 = pd.read_csv('191119_Deuterium_Transfer_Peak_Areas.csv', comment='#')

df1o = pd.read_excel('190926_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df2o = pd.read_excel('191007_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df3o = pd.read_excel('191010_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df4o = pd.read_excel('191015_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df5o = pd.read_excel('191020_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df6o = pd.read_excel('191021_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df7o = pd.read_excel('191022_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df8o = pd.read_excel('191025_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df9o = pd.read_excel('191027_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df10o = pd.read_excel('191031_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')
df11o = pd.read_excel('191106_Deuterium_Transfer_Peak_Areas_old.xlsx', comment='#')

In [3]:
#add a column to each dataframe listing the date
df1['date'] = '190926'
df2['date'] = '191007'
df3['date'] = '191010'
df4['date'] = '191015'
df5['date'] = '191020'
df6['date'] = '191021'
df7['date'] = '191022'
df8['date'] = '191025'
df9['date'] = '191027'
df10['date'] = '191031'
df11['date'] = '191106'
df12['date'] = '191115'
df13['date'] = '191119'

df1o['date'] = '190926'
df2o['date'] = '191007'
df3o['date'] = '191010'
df4o['date'] = '191015'
df5o['date'] = '191020'
df6o['date'] = '191021'
df7o['date'] = '191022'
df8o['date'] = '191025'
df9o['date'] = '191027'
df10o['date'] = '191031'
df11o['date'] = '191106'


#merge data frames into one dataframe
df=df1.append(df2).append(df3).append(df4).append(df5).append(df6).append(df7).append(df8).append(df9).append(df10).append(df11).append(df12).append(df13)
df_old=df1o.append(df2o).append(df3o).append(df4o).append(df5o).append(df6o).append(df7o).append(df8o).append(df9o).append(df10o).append(df11o)

df.tail()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date
43,L,8,D50C24,9.480,9.475,9.525,TIC,1020,0.01,2002,0.02,0.51,U,N,191119
44,L,8,D62C30,12.380,12.335,12.380,TIC,34904,0.30,33043,0.34,1.06,U,N,191119
45,D,8,C18,7.439,7.405,7.500,TIC,11443903,99.74,9614537,99.71,1.19,U,N,191119
46,D,8,D50C24,9.525,9.475,9.525,TIC,3225,0.03,3986,0.04,0.81,U,N,191119
47,D,8,D62C30,12.380,12.335,12.380,TIC,26344,0.23,24262,0.25,1.09,U,N,191119


In [4]:
#function to normalize the peak are of each unique peak using the area of the C18 peak for each sample
def normalize_area(data):
    #check to make sure that the dataframe hasn't already been normalized
    if not {'Normalized Area', 'Hydrocarbon amount'}.issubset(data.columns):
        #Create array of c18 values repeated in groups of three, matching the number of unique peaks for each sample
        c18_Areas = np.repeat(data['Area'].loc[data['Peak ID'] == 'C18'],3).reset_index().drop("index", axis=1)
        
        #Divide the area of each peak in the dataframe by the corresponding c18 peak area
        data['Normalized Area'] = np.divide(data['Area'],c18_Areas['Area'])
        
        #Calculate the hydrocarbon amount, in ng, by multiplying the normalized peak area by 25.
        #25 ng of c18 was injected in each sample (1 microliter of a 25 ng/microliter solution of c18 in hexane)
        data['Hydrocarbon amount'] = data['Normalized Area']*25
        
    return data

In [5]:
df_old = normalize_area(df_old)
df = normalize_area(df)

In [6]:
#Remove any outliers, specified by 'Y' in the Outlier column
df = df.loc[df['Outlier'] == 'N']
df_old = df_old.loc[df_old['Outlier'] == 'N']

In [7]:
#function to create an identifier for each sample in the order (Liometopum or not : Beetle: Type)
def add_identifier(data):
    #check if identifier column already exists
    if not {'Identifier'}.issubset(data.columns):
        
        #create identifier column, populating it with the beetle species used in each well
        data = data.merge(data[['WellNumber','Species','date']].loc[data['Species'] != 'L'].rename(columns={"Species": "Identifier"}).drop_duplicates())
        
        #modify the identifier column, adding a leading 'L' for Liometopum runs and adding a suffix from the 'Type' column
        data['Identifier'] = ((data['Species']=='L')*pd.Series('L',index=data.index)) + data['Identifier'] + data['Type']
        
    return data

# df['Identifier']=''
# for j,date in enumerate(np.unique(df['date'])):
#     df_date = df.loc[df['date']==date]
#     for i in range(8):
#         curSample = df_date.loc[df_date['WellNumber'] == i+1]
#         if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LSU'
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'S')] = 'SU'
                    
#         if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LSL'
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'S')] = 'SL'
        
        
#         if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LPU'
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'P')] = 'PU'
        
#         if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LPL'
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'P')] = 'PL'
            
            
#         if (any((curSample['Species'] == 'D') & (curSample['Type'] == 'U'))):
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LDU'
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'D')] = 'DU'
        
#         if (any((curSample['Species'] == 'D') & (curSample['Type'] == 'L'))):
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LDL'
#             df['Identifier'].loc[(df['date']==date) & (df['WellNumber']==i+1) & (df['Species'] == 'D')] = 'DL'
        

In [8]:
df_old = add_identifier(df_old)
df = add_identifier(df)

In [9]:
df

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date,Normalized Area,Hydrocarbon amount,Identifier
0,L,1,C18,7.435,7.405,7.470,TIC,5589973,93.17,5956210,95.09,0.94,L,N,190926,1.000000,25.000000,LSL
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926,0.041494,1.037348,LSL
2,L,1,D62C30,12.350,12.330,12.375,TIC,177959,2.97,85013,1.36,2.09,L,N,190926,0.031835,0.795885,LSL
3,S,1,C18,7.434,7.410,7.455,TIC,6037087,99.11,6123349,99.37,0.99,L,N,190926,1.000000,25.000000,SL
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926,0.006242,0.156048,SL
5,S,1,D62C30,12.350,12.330,12.375,TIC,16273,0.27,7006,0.11,2.32,L,N,190926,0.002696,0.067388,SL
6,L,2,C18,7.434,7.405,7.455,TIC,6395384,99.69,6376001,99.70,1.00,U,N,190926,1.000000,25.000000,LSU
7,L,2,D50C24,9.525,9.475,9.525,TIC,1607,0.03,756,0.01,2.13,U,N,190926,0.000251,0.006282,LSU
8,L,2,D62C30,12.375,12.330,12.375,TIC,18105,0.28,18434,0.29,0.98,U,N,190926,0.002831,0.070774,LSU
9,S,2,C18,7.435,7.405,7.460,TIC,6036095,99.92,6131401,99.94,0.98,U,N,190926,1.000000,25.000000,SU


In [10]:
def box_and_whisker(data,plt,x_vals,y_vals):
    
    groups = data.groupby(x_vals)
    q1 = groups.quantile(q=0.25)
    q2 = groups.quantile(q=0.5)
    q3 = groups.quantile(q=0.75)
    iqr = q3 - q1
    upper = q3 + 1.5*iqr
    lower = q1 - 1.5*iqr

    # find the outliers for each category
    def outliers(group):
        cat = group.name
        return group[(group[y_vals] > upper.loc[cat][y_vals]) | (group[y_vals] < lower.loc[cat][y_vals])][y_vals]
    out = groups.apply(outliers).dropna()

    # prepare outlier data for plotting, we need coordinates for every outlier.
    if not out.empty:
        outx = []
        outy = []
        for keys in out.index:
            outx.append(keys[0])
            outy.append(out.loc[keys[0]].loc[keys[1]])

    # if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
    qmin = groups.quantile(q=0.00)
    qmax = groups.quantile(q=1.00)
    upper[y_vals] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,y_vals]),upper[y_vals])]
    lower[y_vals] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,y_vals]),lower[y_vals])]

    # stems
    plt.segment(q3.reset_index()[x_vals], upper[y_vals], q3.reset_index()[x_vals], q3[y_vals], line_color="black")
    plt.segment(q3.reset_index()[x_vals], lower[y_vals], q3.reset_index()[x_vals], q1[y_vals], line_color="black")

    #boxes
    plt.vbar(q3.reset_index()[x_vals], 0.7, q2[y_vals], q3[y_vals], fill_color=None, line_color="black")
    plt.vbar(q3.reset_index()[x_vals], 0.7, q1[y_vals], q2[y_vals], fill_color=None, line_color="black")

    # whiskers (almost-0 height rects simpler than segments)
    plt.vbar(q3.reset_index()[x_vals], top=lower[y_vals],bottom=lower[y_vals], width=0.2, line_color="black")
    plt.vbar(q3.reset_index()[x_vals], top=upper[y_vals],bottom=upper[y_vals], width=0.2, line_color="black")
    
    return plt

In [11]:
df_C24 = df.loc[df['Peak ID'] == 'D50C24']
df_C30 = df.loc[df['Peak ID'] == 'D62C30']

df_C24_old = df_old.loc[df_old['Peak ID'] == 'D50C24']

In [20]:
plotting_df=df_C24

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

p = box_and_whisker(plotting_df,p,'Identifier','Hydrocarbon amount')

for index, row in plotting_df[['WellNumber','date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values,
            'hydrocarbon amount': plotting_df['Hydrocarbon amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
#     p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
#            'hydrocarbon amount',
#            source=source,
#            color='black',
#            alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [13]:
plotting_df=df_C24_old

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

p = box_and_whisker(plotting_df,p,'Identifier','Hydrocarbon amount')

for index, row in plotting_df[['WellNumber','date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values,
            'hydrocarbon amount': plotting_df['Hydrocarbon amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
    p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color='black',
           alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [14]:
plotting_df=df_C30

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

p = box_and_whisker(plotting_df,p,'Identifier','Hydrocarbon amount')

for index, row in plotting_df[['WellNumber','date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values,
            'hydrocarbon amount': plotting_df['Hydrocarbon amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
    p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color='black',
           alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [15]:
df_C24['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C24['date'])):
    df_C24.loc[df_C24['date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C24.loc[df_C24['date']==date,'Hydrocarbon amount'].values.reshape(-1, 1)).ravel()

df_C24_old['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C24_old['date'])):
    df_C24_old.loc[df_C24_old['date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C24_old.loc[df_C24_old['date']==date,'Hydrocarbon amount'].values.reshape(-1, 1)).ravel()    

df_C30['Scaled Amount']=0
for _,date in enumerate(np.unique(df_C30['date'])):
    df_C30.loc[df_C30['date']==date,'Scaled Amount'] = preprocessing.MinMaxScaler().fit_transform(df_C30.loc[df_C30['date']==date,'Hydrocarbon amount'].values.reshape(-1, 1)).ravel()

C:\Users\Tom\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Tom\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Tom\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [16]:
df_C24.head()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier,date,Normalized Area,Hydrocarbon amount,Identifier,Scaled Amount
1,L,1,D50C24,9.498,9.475,9.525,TIC,231950,3.87,222378,3.55,1.04,L,N,190926,0.041494,1.037348,LSL,0.891385
4,S,1,D50C24,9.497,9.475,9.525,TIC,37683,0.62,31671,0.51,1.19,L,N,190926,0.006242,0.156048,SL,0.129477
7,L,2,D50C24,9.525,9.475,9.525,TIC,1607,0.03,756,0.01,2.13,U,N,190926,0.000251,0.006282,LSU,0.000000
10,S,2,D50C24,9.525,9.475,9.525,TIC,2344,0.04,1432,0.02,1.64,U,N,190926,0.000388,0.009708,SU,0.002962
13,L,3,D50C24,9.525,9.475,9.525,TIC,4472,0.08,2979,0.05,1.50,U,N,190926,0.000763,0.019070,LSU,0.011056


In [17]:
plotting_df=df_C24

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='scaled D50C24')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for index, row in plotting_df[['WellNumber','date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values,
            'scaled amount': plotting_df['Scaled Amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

# p.output_backend = "svg"
# export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)

In [18]:
plotting_df=df_C24_old

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='scaled D50C24')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for index, row in plotting_df[['WellNumber','date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values,
            'scaled amount': plotting_df['Scaled Amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

# p.output_backend = "svg"
# export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)

In [19]:
plotting_df=df_C30

Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}

np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=800,
                          plot_height=600,
                          title='C30',
                          x_range=Identifiers,
                          y_axis_label='scaled D62C30')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for index, row in plotting_df[['WellNumber','date']].drop_duplicates().iterrows():
        
    data = {'identifier': plotting_df['Identifier'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values,
            'scaled amount': plotting_df['Scaled Amount'].loc[(plotting_df['WellNumber']==row['WellNumber']) & (plotting_df['date']==row['date'])].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
               'scaled amount',
               source=source,
               color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
               alpha=0.6,
               size=7)


p.xgrid.visible = False
p.ygrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

# p.output_backend = "svg"
# export_svgs(p, filename="DCHC_transfer_C24_minmax_scaled.svg")

bokeh.io.show(p)